# Create Best Model Using Uber Dataset

#### Import Dependencies

In [ ]:
# import pandas as pd
# import numpy as np
# import optuna

# from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.ensemble import IsolationForest, RandomForestRegressor,GradientBoostingRegressor, HistGradientBoostingRegressor, AdaBoostRegressor
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from math import radians, sin, cos, sqrt, asin
# from tqdm import tqdm
# from catboost import CatBoostRegressor

# from sklearn.tree import DecisionTreeRegressor, plot_tree
# from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor
# from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [159]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import IsolationForest, RandomForestRegressor,GradientBoostingRegressor, HistGradientBoostingRegressor, AdaBoostRegressor

# model
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge

# metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from tqdm import tqdm

#### Data Loading, Cleaning, Preprocessing

In [95]:
df = pd.read_csv("../data/uber.csv", parse_dates=['pickup_datetime'])
df.drop(['Unnamed: 0','key'], axis=1, inplace=True)
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5


In [96]:
print(f'Inference: The Dataset consists of {df.shape[1]} features & {df.shape[0]} samples.')

Inference: The Dataset consists of 7 features & 200000 samples.


In [97]:
nullValues = pd.DataFrame(df.isnull().sum().sort_values(), columns=['Total Null Values'])
nullValues['Percentage'] = round(nullValues['Total Null Values']/df.shape[0],3)*100
print(nullValues)

                   Total Null Values  Percentage
fare_amount                        0         0.0
pickup_datetime                    0         0.0
pickup_longitude                   0         0.0
pickup_latitude                    0         0.0
passenger_count                    0         0.0
dropoff_longitude                  1         0.0
dropoff_latitude                   1         0.0


In [98]:
df.dropna(inplace=True)

In [99]:
df.isnull().sum()# after clean dataset

fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [100]:
df.duplicated().sum()#checking duplicate values

0

In [101]:
df_delete = df[(df['pickup_latitude']>180) | (df['pickup_latitude']<-180)|
                    (df['dropoff_latitude']>180) | (df['dropoff_latitude']< -180)|
                    (df['pickup_longitude']>90) | (df['pickup_longitude']<-90) |
                    (df['dropoff_longitude']>90) | (df['dropoff_longitude']<-90)]
df_delete.shape

(13, 7)

In [102]:
index_to_delete = df_delete.index
index_to_delete

Index([  4949,  32549,  40908,  48506,  56617,  61793,  75851,  91422, 103745,
       139447, 144253, 161652, 199936],
      dtype='int64')

In [103]:
before = df.shape
df.drop(index_to_delete,inplace=True)
print(f'before :{before[0]},after :{df.shape[0]}')

before :199999,after :199986


In [104]:
df['year'] = df.pickup_datetime.dt.year
df['month'] = df.pickup_datetime.dt.month
df['weekday'] = df.pickup_datetime.dt.weekday
df['hour'] = df.pickup_datetime.dt.hour

In [105]:
df['Monthly_Quarter'] = df.month.map({1:'Q1',2:'Q1',3:'Q1',4:'Q2',5:'Q2',6:'Q2',7:'Q3',
                                      8:'Q3',9:'Q3',10:'Q4',11:'Q4',12:'Q4'})

df['Hourly_Segments'] = df.hour.map({0:'H1',1:'H1',2:'H1',3:'H1',4:'H2',5:'H2',6:'H2',7:'H2',8:'H3',
                                     9:'H3',10:'H3',11:'H3',12:'H4',13:'H4',14:'H4',15:'H4',16:'H5',
                                     17:'H5',18:'H5',19:'H5',20:'H6',21:'H6',22:'H6',23:'H6'})

In [106]:
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,Monthly_Quarter,Hourly_Segments
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,2015,5,3,19,Q2,H5
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2009,7,4,20,Q3,H6
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,2009,8,0,21,Q3,H6
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,2009,6,4,8,Q2,H3
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,2014,8,3,17,Q3,H5


In [107]:
df.drop(['pickup_datetime','month', 'hour',], axis=1, inplace=True)

##### Calculate Distance

In [108]:
from math import radians, sin, cos, sqrt, asin

def distance_transform(longitude1, latitude1, longitude2, latitude2):
    travel_dist = []
    
    for pos in range(len(longitude1)):
        long1,lati1,long2,lati2 = map(radians,[longitude1[pos],latitude1[pos],longitude2[pos],latitude2[pos]])
        dist_long = long2 - long1
        dist_lati = lati2 - lati1
        a = sin(dist_lati/2)**2 + cos(lati1) * cos(lati2) * sin(dist_long/2)**2
        c = 2 * asin(sqrt(a))*6371
        travel_dist.append(c)
       
    return travel_dist

In [109]:
df['distance_traveled']=distance_transform(df['pickup_longitude'].to_numpy(),
                                  df['pickup_latitude'].to_numpy(),
                                  df['dropoff_longitude'].to_numpy(),
                                  df['dropoff_latitude'].to_numpy())
## This Distance is in kilometers
df['distance_traveled'] = df['distance_traveled'].round(3)

In [110]:
df[df['fare_amount']<= 0].shape

(22, 11)

In [111]:
df_fare_delete = df[df['fare_amount']<=0]
df_fare_delete.shape

(22, 11)

In [112]:
old_shape = df.shape
index_to_delete_fare = df_fare_delete.index
df.drop(index_to_delete_fare,inplace=True)
print(f"Number of items removed : {old_shape[0] - df.shape[0]}")

Number of items removed : 22


In [113]:
df['passenger_count'].value_counts()

passenger_count
1      138404
2       29423
5       14004
3        8878
4        4275
6        4271
0         708
208         1
Name: count, dtype: int64

In [114]:
old_shape = df.shape
index_to_delete_passenger = df[df['passenger_count']==df['passenger_count'].max()].index
df.drop(index_to_delete_passenger,inplace=True)
print(f"After drop {old_shape[0] - df.shape[0]}")

After drop 1


In [115]:
# now the large passenger count is removed 208 column
df['passenger_count'].value_counts()

passenger_count
1    138404
2     29423
5     14004
3      8878
4      4275
6      4271
0       708
Name: count, dtype: int64

In [116]:
# here we will replace 0 with 1 count
df['passenger_count']=np.where(df['passenger_count']==0,1,df['passenger_count'])

In [117]:
df['passenger_count'].value_counts()

passenger_count
1    139112
2     29423
5     14004
3      8878
4      4275
6      4271
Name: count, dtype: int64

In [118]:
# removing zero distance travel
index_to_delete_distance = df[df['distance_traveled'] == 0].index
df.drop(index_to_delete_distance,inplace=True)
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,distance_traveled
count,194247.000000,194247.000000,194247.000000,194247.000000,194247.000000,194247.000000,194247.000000,194247.000000,194247.000000
mean,11.349822,-73.826480,40.646902,-73.837556,40.651579,1.687413,2011.747420,3.048577,21.060440
std,9.722955,3.660507,2.931640,3.536295,2.900095,1.303961,1.859941,1.946791,384.091421
min,0.010000,-89.933333,-74.015515,-75.458979,-74.015750,1.000000,2009.000000,0.000000,0.001000
25%,6.000000,-73.992269,40.736348,-73.991589,40.735220,1.000000,2010.000000,1.000000,1.283000
50%,8.500000,-73.982116,40.753253,-73.980539,40.753708,1.000000,2012.000000,3.000000,2.185000
75%,12.500000,-73.968400,40.767508,-73.965423,40.768314,2.000000,2013.000000,5.000000,3.961000
max,499.000000,40.808425,48.018760,40.831932,45.031598,6.000000,2015.000000,6.000000,8782.899000


In [119]:
len(df[df.distance_traveled < 0.1])

1081

In [120]:
# removing 100m distance traveled
before = df.shape
df.drop(df[df.distance_traveled < 0.1].index, inplace=True)
print(f"before :{before[0]},after :{df.shape[0]}")

before :194247,after :193166


In [121]:
df_delete_lat_long = df[(df['pickup_longitude']==0) | (df['pickup_latitude']==0) | (df['dropoff_longitude']==0) | (df['dropoff_latitude']==0) ]
df_delete_lat_long

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,Monthly_Quarter,Hourly_Segments,distance_traveled
346,15.50,0.000000,0.000000,-73.979805,40.786030,1,2015,3,Q1,H5,8666.398
1067,52.00,-73.781095,40.645015,0.000000,0.000000,1,2014,6,Q1,H6,8647.036
1526,2.50,-74.001849,40.715156,0.000000,0.000000,3,2014,0,Q2,H4,8666.772
2547,10.10,0.000000,0.000000,-73.953210,40.803528,2,2011,1,Q3,H6,8664.557
3045,15.00,0.000000,0.000000,-73.843777,40.739255,1,2013,1,Q1,H1,8654.177
...,...,...,...,...,...,...,...,...,...,...,...
196967,57.33,0.000000,0.000000,-73.789045,40.655135,2,2014,2,Q3,H3,8647.904
197468,6.90,0.000000,0.000000,-73.980827,40.747133,5,2011,2,Q1,H5,8665.686
197863,7.00,-73.962190,40.759158,0.000000,0.000000,1,2014,1,Q4,H6,8664.389
198567,23.50,-73.968115,40.801455,0.000000,0.000000,2,2013,0,Q4,H1,8665.747


In [122]:
old_shape = df.shape
index_to_delete_lat_long = df[(df['pickup_longitude']==0) | (df['pickup_latitude']==0) | (df['dropoff_longitude']==0) | (df['dropoff_latitude']==0) ].index
df.drop(index_to_delete_lat_long,inplace=True)
print(f"After drop {old_shape[0] - df.shape[0]}")

After drop 379


#### Encoding Categorical Features

In [123]:
Hour_encoder = LabelEncoder()
Month_encoder = LabelEncoder()

df.Monthly_Quarter = Month_encoder.fit_transform(df.Monthly_Quarter)
df.Hourly_Segments = Hour_encoder.fit_transform(df.Hourly_Segments)

Hour_Segments_decoded = Hour_encoder.inverse_transform(df.Hourly_Segments)
print("Hour Segments Decoded Categories:", Hour_Segments_decoded)

Month_Segments_decoded = Month_encoder.inverse_transform(df.Monthly_Quarter)
print("Decoded Categories:", Month_Segments_decoded)

Hour Segments Decoded Categories: ['H5' 'H6' 'H6' ... 'H1' 'H4' 'H2']
Decoded Categories: ['Q2' 'Q3' 'Q3' ... 'Q2' 'Q2' 'Q2']


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192787 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   fare_amount        192787 non-null  float64
 1   pickup_longitude   192787 non-null  float64
 2   pickup_latitude    192787 non-null  float64
 3   dropoff_longitude  192787 non-null  float64
 4   dropoff_latitude   192787 non-null  float64
 5   passenger_count    192787 non-null  int64  
 6   year               192787 non-null  int32  
 7   weekday            192787 non-null  int32  
 8   Monthly_Quarter    192787 non-null  int64  
 9   Hourly_Segments    192787 non-null  int64  
 10  distance_traveled  192787 non-null  float64
dtypes: float64(6), int32(2), int64(3)
memory usage: 16.2 MB


In [125]:
counter = 0
rows,columns = df.shape
df.drop_duplicates(inplace=True)
df.drop(['pickup_latitude','pickup_longitude',
         'dropoff_latitude','dropoff_longitude'],axis=1)
if df.shape==(rows,columns):
    print('Inference: The dataset doesn\'t have any duplicates')
else:
    print(f'Inference: Number of duplicates dropped fixed ---> {rows-df.shape[0]}')

Inference: The dataset doesn't have any duplicates


#### Feature Selection

In [126]:
features =  ['passenger_count', 'year',
             'weekday', 'Monthly_Quarter', 
             'Hourly_Segments', 'distance_traveled']

target = 'fare_amount'

In [127]:
# Splitting Data into X and Y
X, y = df[features], df[target]

In [129]:
selector = SelectKBest(score_func=f_regression, k=6)
X_new = selector.fit_transform(X, y)

selected_feature_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_feature_indices]

print("Selected features:\n")
print('\n'.join(map(str, selected_features)))

Selected features:

passenger_count
year
weekday
Monthly_Quarter
Hourly_Segments
distance_traveled


### Data Splitting into Train and test set, Data Scaling and Model Training

In [150]:
# from sklearn.model_selection import train_test_split

In [151]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=False)

In [152]:
# manual splitting up to 2015 data
train, test = df[df.year < 2015], df[df.year == 2015]

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [153]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (179407, 6)
y_train shape: (179407,)
X_test shape: (13380, 6)
y_test shape: (13380,)


In [154]:
# transforming the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((179407, 6), (179407,), (13380, 6), (13380,))

In [162]:
models_collection = {
    'Random_Forest_1': RandomForestRegressor(),
    'Decision_tree_1': DecisionTreeRegressor(),
    'Linear_Regression_1': LinearRegression(),
    'XGBoost_1': XGBRegressor(),
    'Lasso_1': Lasso(),
    'Gradient_Boost_1' : GradientBoostingRegressor(),
    'High_Gradient_Boost_1' : HistGradientBoostingRegressor(),
}

In [163]:
def eval_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, r2

In [164]:
train_test_results = {}

for model_name, model in tqdm(models_collection.items(), desc='Training Models'):
    model.fit(X_train, y_train.ravel())

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    trained_data_mae, trained_data_mse, trained_data_r2 = eval_model(y_train, y_train_pred)
    test_data_mae, test_data_mse, test_data_r2 = eval_model(y_test, y_test_pred)

    train_test_results[model_name] = {
        'Train': {
            'Train_MAE': trained_data_mae,
            'Train_R2': trained_data_r2,
            'Train_mse': trained_data_mse
        },
        'Test': {
            'Test_MAE': test_data_mae,
            'Test_R2': test_data_r2,
            'Test_mse': test_data_mse
        }
    }

Training Models: 100%|██████████| 7/7 [01:26<00:00, 12.41s/it]


In [166]:
print('='*80)
print('{:<25s} {:<8s} {:<8s} {:<8s} {:<8s}'.format('Model', 'Train_MSE', 'Test_MSE', 'Train_R2', 'Test_R2'))
print('='*80)
for model_name, model_results in train_test_results.items():
    print('{:<25s} {:<9.3f} {:<9.3f} {:<9.3f} {:<9.3f}'.format(
        model_name,
        model_results['Train']['Train_mse'],
        model_results['Test']['Test_mse'],
        model_results['Train']['Train_R2'],
        model_results['Test']['Test_R2'],
    ))
print('='*80)

Model                     Train_MSE Test_MSE Train_R2 Test_R2 
Random_Forest_1           2.352     17.788    0.973     0.855    
Decision_tree_1           0.062     41.664    0.999     0.661    
Linear_Regression_1       86.265    122.500   0.018     0.003    
XGBoost_1                 11.418    15.400    0.870     0.875    
Lasso_1                   87.542    124.973   0.004     -0.017   
Gradient_Boost_1          14.090    14.740    0.840     0.880    
High_Gradient_Boost_1     15.314    15.402    0.826     0.875    
